#Importo librerie

In [1]:
import pandas as pd
import numpy as np
import concurrent.futures
import time
from requests.exceptions import ReadTimeout

In [2]:
!pip install -U -q PyDrive

In [79]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#Autenticazione Spotify API

In [4]:
!pip install spotipy

In [5]:
# autenticazione Spotify API con spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
auth_manager = SpotifyClientCredentials(client_id='caf57b996b464996bff50ab59186f265', client_secret='0bfcdbff8015426cae855b56b692f69b')
sp = spotipy.Spotify(auth_manager=auth_manager, requests_timeout=10)

#Importo dataset

In [80]:
# autenticazione google drive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [7]:
# importo prima parte del dataset --> 7087 datapoints

drive.CreateFile({'id':'1YAAvPUaPeBIddkuVzWgw7fhPPcT2uJTY'}).GetContentFile('billboard_dataset_unique.csv')
df_billboard = pd.read_csv("billboard_dataset_unique.csv").drop('Unnamed: 0',axis=1).iloc[:7087]

drive.CreateFile({'id':'1eOqgPk_izGXKIT5y6KfqPkmKWqBonVc0'}).GetContentFile('dataset2_X_billboard.csv')
df_songs = pd.read_csv("dataset2_X_billboard.csv").drop('Unnamed: 0',axis=1)


# df_billboard.iloc[:7087]

# df_billboard.iloc[7087:14174]

# df_billboard.iloc[14174:21261]

# df_billboard.iloc[21261:]

In [ ]:
df_billboard.head()

,title,artist,weeks
0,Mood,24kGoldn Featuring iann dior,22
1,Positions,Ariana Grande,11
2,Blinding Lights,The Weeknd,57
3,Holy,Justin Bieber Featuring Chance The Rapper,16
4,Go Crazy,Chris Brown & Young Thug,35


In [ ]:
df_billboard.shape

(7087, 3)

#Definizione funzioni

In [8]:
def print_exec_time(start):
  print("Esecuzione completata in %.4f secondi" % (time.perf_counter()-start))

In [9]:
# funzione che effettua ricerca con Spotify API considerando i casi in cui nel campo 'artist' siano presenti più artisti (featuring)

def search_fix(artist, title):
  artist_separators = ['%%%', ' Featuring', ' featuring', ' feat.', ' Feat.', ' feat', ' Feat', ' &', ' x', ' X', ' with', ' With', ', ', '/', ' duet', ' Duet', '+', ' and']
  title_separators = ['%%%', ' (']
  title_fix = ["%%%", "'s", "'"]

  id = None

  for x in artist_separators:
    for y in title_separators:
      for z in title_fix:
        try:
          id = sp.search(q='artist:'+artist.split(x)[0]+' track:'+title.split(y)[0].replace(z, ''), type='track', limit=1)['tracks']['items'][0]['id']
        except IndexError:
          pass
        if(id != None):
          break
      if(id != None):
        break
    if(id != None):
      break

  return id

In [10]:
# funzione che prendendo una singola riga del Billboard dataset restituisce una lista con id, artista e titolo
# --> in caso di errore l'id viene impostato a None

def get_id(row):

  artist = row[1]
  title = row[0]

  print("fetching id for %s by %s ..." % (title, artist))

  try:
    try:
      id = sp.search(q='artist:'+artist+' track:'+title, type='track', limit=1)['tracks']['items'][0]['id']
    except IndexError:
      id = search_fix(artist, title)
  except ReadTimeout:
    id = None
      
  if(id == None):
    print('--> [error] %s by %s' % (title, artist))

  return [id, artist, title]

In [20]:
# funzione che, preso un id, restituisce un array con le features (audio e non) della traccia corrispondente

def get_features(id):
  print("fetching features for id: %s" % id)

  # audio features
  danceability = []
  energy = []
  key = []
  loudness =[]
  mode = []
  speechiness = []
  acousticness = []
  instrumentalness = []
  liveness = []
  valence = []
  tempo = []
  duration_ms = []

  audio_features_array = [danceability, energy, key, loudness, mode, speechiness,
      acousticness, instrumentalness, liveness, valence, tempo, duration_ms]

  # altre features
  release_date = []
  explicit = []

  release_date.append(sp.track(id)['album']['release_date'])
  explicit.append(sp.track(id)['explicit'])

  audio_features = sp.audio_features(id)[0]

  try:
    # rimuovo campi non necessari
    to_remove = ['type', 'id', 'uri', 'track_href', 'analysis_url', 'time_signature']
    for rmv in to_remove:
        audio_features.pop(rmv)

    for i, feature in enumerate(audio_features.keys()):
        audio_features_array[i].append(audio_features[feature])

  except AttributeError:
    print("--> [error] id = %s" % id)

    for i in range(12):
        audio_features_array[i].append(None)


  audio_features_array.append(release_date)
  audio_features_array.append(explicit) 

  return audio_features_array

#Integrazione dataset

##Recupero gli id del Billboard dataset

In [ ]:
time_0 = time.perf_counter()

with concurrent.futures.ProcessPoolExecutor() as executor:
  results = executor.map(get_id, df_billboard.values.tolist())
  
  output = []
  for result in results:
    output.append(result)

print_exec_time(time_0)

Output streaming troncato alle ultime 5000 righe.
fetching id for Party Monster by The Weeknd ...
fetching id for Nothings Into Somethings by Drake ...
fetching id for Let Me Love You by DJ Snake Featuring Justin Bieber ...
fetching id for Lose You by Drake ...
--> [error] The Heart Part 4 by Kendrick Lamar
fetching id for Since Way Back by Drake Featuring PARTYNEXTDOOR ...
fetching id for Make Me (Cry) by Noah Cyrus Featuring Labrinth ...
fetching id for Skepta Interlude by Drake ...
fetching id for Kill A Word by Eric Church Featuring Rhiannon Giddens ...
fetching id for Trap Trap Trap by Rick Ross Featuring Young Thug & Wale ...
fetching id for Heathens by twenty one pilots ...
fetching id for Black Beatles by Rae Sremmurd Featuring Gucci Mane ...
fetching id for Caroline by Amine ...
fetching id for Regret In Your Tears by Nicki Minaj ...
fetching id for Better Man by Little Big Town ...
fetching id for Changed It by Nicki Minaj & Lil Wayne ...
fetching id for Chanel by Frank Ocean

In [ ]:
# creo backup del billboard dataset
df_billboard_bak = df_billboard.copy()

In [ ]:
# inserisco gli id ottenuti in una nuova colonna nel df_billboard
ids = np.array(output)[:,0]
df_billboard.insert(0, 'id', ids)

In [ ]:
# calcolo percentuale di canzoni trovate
found_id = df_billboard.id.count()
x = (found_id / df_billboard.title.count()) * 100
print("Found ids = %d%%" % x)

Found ids = 97%


In [ ]:
# esporto su google drive
from google.colab import drive

# mounts the google drive to Colab Notebook
drive.mount('/content/drive',force_remount=True)

df_billboard.to_csv('/content/drive/My Drive/Colab Notebooks/datasets/billboard+ids_0.csv')

Mounted at /content/drive


##Recupero audio features del Billboard dataset

In [81]:
# reimporto dataset billboard (con ids) + dataset principale
"""
drive.CreateFile({'id':'1fZzuYu-HXKP9HUeio-FL9P4eNygOQ0qq'}).GetContentFile('billboard+ids_0.csv') 
df_billboard = pd.read_csv("billboard+ids_0.csv").drop('Unnamed: 0',axis=1)

drive.CreateFile({'id':'1eOqgPk_izGXKIT5y6KfqPkmKWqBonVc0'}).GetContentFile('dataset2_X_billboard.csv')
df_songs = pd.read_csv("dataset2_X_billboard.csv").drop('Unnamed: 0',axis=1)
"""

In [82]:
# elimino valori nulli (= id non trovati)
df_billboard = df_billboard.dropna()

# creo lista con gli id del dataset billboard
ids = list(df_billboard.id.array)

In [15]:
# creo lista degli id che non sono presenti nel dataset principale
time_0 = time.perf_counter()
ids_new = [id for id in ids if id not in list(df_songs.id.array)]
print_exec_time(time_0)

Esecuzione completata in 1634.0306 secondi


In [37]:
time_0 = time.perf_counter()

with concurrent.futures.ProcessPoolExecutor() as executor:
  results = executor.map(get_features, ids_new)

  output = []
  for result in results:
    output.append(result)

print_exec_time(time_0)

fetching features for id: 463CkQjx2Zk1yXoBuierM9
fetching features for id: 31qCy5ZaophVA81wtlwLc4
fetching features for id: 6Im9k8u9iIzKMrmV7BWtlF
fetching features for id: 7a53HqqArd4b9NF4XAmlbI
fetching features for id: 0A1hoCfMLkiAgvhWkkucJa
fetching features for id: 4fKiuIavbsoTRZ6R7xuAyT
fetching features for id: 0lx2cLdOt3piJbcaXIV74f
fetching features for id: 3YJJjQPAbDT7mGpX3WtQ9A
fetching features for id: 2gMXnyrvIjhVBUZwvLZDMP
fetching features for id: 4saklk6nie3yiGePpBwUoc
fetching features for id: 7ytR5pFWmSjzHJIeQkgog4
fetching features for id: 5OELUCYgOHKFAvCERnAvfS
fetching features for id: 0E4Y1XIbs8GrAT1YqVy6dq
fetching features for id: 1xQ6trAsedVPCdbtDAmk0c
fetching features for id: 4Iedi94TIaB2GGb1nMB68v
fetching features for id: 6UelLqGlWMcVH1E5c4H7lY
fetching features for id: 4aarlAfLKVCTxUDNgbwhjH
fetching features for id: 2bV4NwncWMnVvcWyU2yxwB
fetching features for id: 4obHzpwGrjoTuZh2DItEMZ
fetching features for id: 15C4TnrrVdym7UykxQIOTZ
fetching features fo

#Inserisco audio features nel dataset Billboard

In [ ]:
num_datapoints = np.array(output).shape[0]

output = np.array(output).reshape((num_datapoints,14))

In [94]:
# creo backup df_billboard
df_billboard_bak = df_billboard.copy()

# filtro dataset billboard tenendo solo id nell'array 'ids_new', quindi quelli che non sono presenti nel dataset principale
df_billboard = df_billboard[df_billboard.id.isin(ids_new)]

In [96]:
to_insert = ['danceability',
    'energy',
    'key',
    'loudness',
    'mode',
    'speechiness',
    'acousticness',  
    'instrumentalness',
    'liveness',
    'valence',
    'tempo',
    'duration_ms',
    'release_date',
    'explicit']

for i, col in enumerate(output.T):
    df_billboard.insert(4, to_insert[i], col)

In [98]:
# converto colonna 'release_date' in tipo datetime
df_billboard.release_date = pd.to_datetime(df_billboard.release_date,format="%Y-%m-%d",exact=False)

In [99]:
# inserisco colonna 'year'
year = df_billboard['release_date'].apply(lambda x: int(x.year))
df_billboard.insert(6, 'year', year)

In [103]:
# inserisco colonna 'popularity' --> nb: inizializzo a 0 perchè verrà rimossa
df_billboard.insert(17, 'popularity', np.zeros(df_billboard.shape[0]))

In [105]:
# inserisco colonna 'hit'
hit = np.ones(df_billboard.shape[0])
df_billboard.insert(3, 'hit', hit)
df_billboard.hit = df_billboard.hit.apply(int)

In [107]:
df_billboard.head()

,id,title,artist,hit,weeks,explicit,release_date,year,duration_ms,tempo,valence,liveness,instrumentalness,acousticness,speechiness,mode,loudness,key,popularity,energy,danceability
5,31qCy5ZaophVA81wtlwLc4,Anyone,Justin Bieber,1,1,False,2021-01-01,2021,190779,115.884,0.584,0.113,3.13e-06,0.181,0.0345,1,-8.026,2,0.0,0.538,0.686
6,463CkQjx2Zk1yXoBuierM9,Levitating,Dua Lipa Featuring DaBaby,1,14,False,2020-10-01,2020,203064,102.977,0.915,0.0674,0,0.00883,0.0601,0,-3.787,6,0.0,0.825,0.702
12,6Im9k8u9iIzKMrmV7BWtlF,34+35,Ariana Grande,1,10,True,2020-10-30,2020,173711,109.978,0.485,0.248,0,0.237,0.094,1,-6.476,0,0.0,0.585,0.83
14,7a53HqqArd4b9NF4XAmlbI,Kings & Queens,Ava Max,1,22,False,2020-09-18,2020,162399,129.857,0.457,0.124,0,0.00786,0.0405,0,-4.057,1,0.0,0.69,0.637
15,0A1hoCfMLkiAgvhWkkucJa,Body,Megan Thee Stallion,1,7,True,2020-11-20,2020,171067,94.066,0.469,0.107,0,0.0166,0.241,0,-6.1,8,0.0,0.732,0.914


#Esporto

In [109]:
# esporto in google drive
from google.colab import  drive

# mounts the google drive to Colab Notebook
drive.mount('/content/drive',force_remount=True)

df_billboard.to_csv('/content/drive/My Drive/Colab Notebooks/datasets/billboard+features_0.csv')

Mounted at /content/drive
